In [1]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [2]:
# # Uninstall versi TensorFlow yang sekarang
# !pip uninstall -y tensorflow

# # Install versi TensorFlow yang diinginkan
# !pip install tensorflow==2.12.0

# # Install versi Keras terbaru
# pip install keras --upgrade

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install transformers tensorflow
print("\n")
!pip install Sastrawi



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.7 MB/s eta 0:00:00


In [4]:
import re
import keras
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [5]:
# Load new_kamusalay.csv
path_kamusalay = "/content/drive/MyDrive/Colab Notebooks/Tugas Akhir/new_kamusalay.csv"
kamus_df = pd.read_csv(path_kamusalay, encoding='latin1', header=None, names=['slang', 'formal'])
kamus_dict = dict(zip(kamus_df['slang'], kamus_df['formal']))

# Stopword Remover
factory = StopWordRemoverFactory()
stopword_remover = factory.create_stop_word_remover()

# BERT Tokenizer dan Model
model_name = 'cahya/bert-base-indonesian-522M'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)

# Load model Bi-GRU yang sudah kamu latih
bi_gru_model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Tugas Akhir/Bi-GRU.keras')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/545M [00:00<?, ?B/s]

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [6]:
# === Fungsi Standarisasi ===
def standarize_text(text):
    words = text.split()
    new_words = [kamus_dict.get(word, word) for word in words]
    return ' '.join(new_words)

# === Fungsi Preprocessing ===
def preprocess_text(text):
    if pd.isnull(text):
        return ''

    text = re.sub(r'^RT\s+', '', text)
    text = re.sub(r'\bUSER\b', '', text)
    text = text.lower()
    text = re.sub(r'(\\x[0-9a-fA-F]{2})+', '', text)
    text = re.sub(r'&[a-zA-Z]+;', '', text)
    text = re.sub(r'\\n', ' ', text)
    text = re.sub(r'[!"$%&\'()*+,\-./:;<=>?@\[\\\]^{|}~_#]', ' ', text)
    text = standarize_text(text)
    text = stopword_remover.remove(text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# === Fungsi Tokenisasi untuk 1 Teks ===
def get_bert_embedding(single_text, max_len=40):
    encoded = tokenizer(single_text, return_tensors='tf', padding='max_length', truncation=True, max_length=max_len)
    output = bert_model(encoded)
    return output.last_hidden_state.numpy()  # bentuk: (1, 40, 768)

# === Fungsi Prediksi Lengkap ===
def predict_tweet(text):
    clean_text = preprocess_text(text)
    bert_vector = get_bert_embedding(clean_text)  # shape: (1, 40, 768)
    prediction = bi_gru_model.predict(bert_vector)
    return prediction  # biasanya bentuknya (1, 9) dengan probabilitas per labe

In [10]:
# Input Teks Twitter
tweet = 'USER Awaasss... Jgn sampe beritanya sampe ke akhi gerung! Ntar susah nyari kosakatany. klu bcr aja micingin mata, bibir monyong kesamping, suara keluar ky lg nhn kentut.. "politik kotor skl jmn ini!". Klu tahu brt ini.. Plng Sambil ky nhn ngeden.."dungu!". \xf0\x9f\x98\x8b'

# Prediksi
hasil = predict_tweet(tweet)

# Label prediksi
label_names = [
    "HS", "Abusive", "HS_Individual", "HS_Group", "HS_Religion",
    "HS_Race", "HS_Physical", "HS_Gender", "HS_Other"
]

# Probabilitas dan biner
probabilities = hasil[0]
binary_output = (probabilities >= 0.5).astype(int)

# Buat DataFrame tanpa index
df_result = pd.DataFrame({
    "Label": label_names,
    "Probabilitas": probabilities,
    "Prediksi Biner": binary_output
})

# Tampilkan tanpa index
print(df_result.to_string(index=False))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
        Label  Probabilitas  Prediksi Biner
           HS      0.989088               1
      Abusive      0.999985               1
HS_Individual      0.932381               1
     HS_Group      0.002336               0
  HS_Religion      0.005059               0
      HS_Race      0.000161               0
  HS_Physical      0.990626               1
    HS_Gender      0.037941               0
     HS_Other      0.015976               0
